1. Qual a probabilidade de uma música não ser explícita ou não possuir dançabilidade entre 0,5 e 0,6, nem ter menos de 3 minutos de duração.

In [6]:
import csv

data = open("data/spotify-tracks-genre.csv", mode="r" ,encoding="utf8")
data = csv.DictReader(data)
data = list(data)

tamanho = len(data)

popularidades_ritmos = {}

for linha in data:
  if linha["track_genre"] in popularidades_ritmos:
    popularidades_ritmos[linha["track_genre"]] += int(linha["popularity"])
  else:
    popularidades_ritmos[linha["track_genre"]] = int(linha["popularity"])

  maior = {"_": 0}

  for key in popularidades_ritmos:
    valor = list(maior.items())[0][1]

    if popularidades_ritmos[key] > valor:
      maior = {key:popularidades_ritmos[key] }

print(maior)

{'pop-film': 59283}


2. Qual a probabilidade de uma música ser popular (>P70) se ela for do ritmo mais popular?

Qual a proporção que as músicas do ritmo mais popular representam dentre as músicas populares?


In [9]:
import csv
import numpy

data = open("data/spotify-tracks-genre.csv", mode="r" ,encoding="utf8")
data = csv.DictReader(data)
data = list(data)

tamanho = len(data)

valores_pp = [int(linha["popularity"]) for linha in data]

nivel_pp = float(numpy.percentile(valores_pp, 70))

populares = 0
de_ritmopp = 0
ocorr_desej = 0

for linha in data:
  prim, seg = False, False

  if int(linha["popularity"]) > nivel_pp:
    prim = True
    populares +=1

  if linha["track_genre"] == "pop-film":
    seg = True
    de_ritmopp += 1

  if prim and seg:
    ocorr_desej += 1

probabilidade = ocorr_desej/de_ritmopp
proporcao = de_ritmopp/populares

print(f"A probabilidade é de: {probabilidade*100:.3f}%")
print(f"A proporção é de: {proporcao*100:.3f}%")

A probabilidade é de: 94.600%
A proporção é de: 3.055%


3. Qual a probabilidade de uma música não ser explícita ou não possuir dançabilidade entre 0,5 e 0,6, nem ter menos de 3 minutos de duração.


In [5]:
import csv

data = open("data/spotify-tracks-genre.csv", mode="r" ,encoding="utf8")
data = csv.DictReader(data)
data = list(data)

tamanho = len(data)
count_1 = 0
count_2 = 0

for i in range(tamanho):
  linha = data[i]

  e_explicito = linha["explicit"] != "FALSE"

  dancabilidade = float(linha["danceability"].replace(",", "."))
  danca_entre = dancabilidade > 0.5 and dancabilidade < 0.6

  duracao = float(linha["duration_ms"]) / 60000
  tem_3_minutos = duracao <= 3

  if (not e_explicito or not danca_entre) and not tem_3_minutos:
    count_1 += 1


print(round((count_1 / tamanho) * 100, 2), "%")

70.57 %


4. Dentre os traços (duration_ms, danceability, energy, key, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo) qual é mais provável de indicar uma música popular caso:

a. esteja abaixo do primeiro quartil?

b. esteja acima do terceiro quartil?

c. esteja dentro de ±25% do segundo quartil?


In [7]:
import csv
from statistics import quantiles

data = open("data/spotify-tracks-genre.csv", mode="r" ,encoding="utf8")
data = csv.DictReader(data)
data = list(data)

tamanho = len(data)

keys = {"duration_ms": [], "danceability": [], "energy": [], "key": [], "loudness": [], "speechiness": [], "acousticness": [],
"instrumentalness": [], "liveness": [], "valence": [], "tempo": [],}

maior = 0
maiores = []

for linha in data:
  for key in keys:
    keys[key].append(float(linha[key].replace(",", ".")))

quartis = {}

for key in keys:
  valores = keys[key]
  quartil_1, quartil_2, quartil_3 = quantiles(valores, n=4 )

  quartis[key] = []

  casos_abaixo_1_quartil = 0
  casos_abaixo_1_quartil_e_popular = 0

  casos_intervalo_2 = 0
  casos_intervalo_2_e_popular = 0

  casos_abaixo_3_quartil = 0
  casos_abaixo_3_quartil_e_popular = 0

  limit_inf = quartil_2 - 0.25 * (quartil_2 - quartil_1)
  limit_sup = quartil_2 + 0.25 * (quartil_3 - quartil_2)

  for linha in data:
    valor = float(linha[key].replace(",", "."))

    # primeiro quartil
    if valor < quartil_1 :
      casos_abaixo_1_quartil += 1
      if int(linha["popularity"]) > 47:
        casos_abaixo_1_quartil_e_popular += 1

    # terceiro:
    if valor > quartil_3 :
        casos_abaixo_3_quartil += 1
        if int(linha["popularity"]) > 47:
          casos_abaixo_3_quartil_e_popular += 1

    # segundo
    if valor > limit_inf and valor < limit_sup:
      casos_intervalo_2 += 1
      if int(linha["popularity"]) > 47:
        casos_intervalo_2_e_popular += 1

  quartis[key].append([casos_abaixo_1_quartil, casos_abaixo_1_quartil_e_popular])
  quartis[key].append([casos_abaixo_3_quartil, casos_abaixo_3_quartil_e_popular])
  quartis[key].append([casos_intervalo_2, casos_intervalo_2_e_popular])

resposta = [("a", 0), ("a", 0), ("a", 0)]

for quartil in quartis:
  dados = quartis[quartil]

  j = 0
  for probabilidade in dados:

    if probabilidade[0] != 0 or probabilidade[1] != 0:
      probabilidade = probabilidade[1] / probabilidade[0]
      probabilidade = round(probabilidade * 100, 2)

      if probabilidade > resposta[j][1]:
        resposta[j] = (quartil, probabilidade)

    j += 1

i = 0
letras = ["a", "b", "c"]

for letra in resposta:
  print(f"{letras[i]}. Traço: {letra[0]}; Porcentagem: {letra[1]:.3f}%")
  i += 1

a. Traço: speechiness; Porcentagem: 32.050%
b. Traço: loudness; Porcentagem: 30.300%
c. Traço: duration_ms; Porcentagem: 32.940%
